In [29]:
import numpy as np
import cv2
from imutils.perspective import four_point_transform
from imutils import contours
from crossword_utils import *

def debug(key, val, format="%-10s %s"):
    print (format % (str(key)+':', val))

def sort_rect_cells(cells):
    # sort bounding boxes
    rect = [o["rect"] for o in cells]
    # sort all rect by their y
    rect.sort(key=lambda b: b[1])
    # initially the line bottom is set to be the bottom of the first rect
    line_bottom = rect[0][1]+rect[0][3]-1
    line_begin_idx = 0
    for i in range(len(rect)):
        # when a new box's top is below current line's bottom
        # it's a new line
        if rect[i][1] > line_bottom:
            # sort the previous line by their x
            rect[line_begin_idx:i] = sorted(rect[line_begin_idx:i], key=lambda b: b[0])
            line_begin_idx = i
        # regardless if it's a new line or not
        # always update the line bottom
        line_bottom = max(rect[i][1]+rect[i][3]-1, line_bottom)

    # sort the last line
    rect[line_begin_idx:] = sorted(rect[line_begin_idx:], key=lambda b: b[0])

    sorted_rect = image.copy()
    for (index, r) in enumerate(rect):
        label_bounding_rect(sorted_rect, r, index)

    show_wait_destroy("sorted rect", sorted_rect)

    return rect

def sort_contours(contours, x_axis_sort='LEFT_TO_RIGHT', y_axis_sort='TOP_TO_BOTTOM'):
    # initialize the reverse flag
    x_reverse = False
    y_reverse = False
    if x_axis_sort == 'RIGHT_TO_LEFT':
        x_reverse = True
    if y_axis_sort == 'BOTTOM_TO_TOP':
        y_reverse = True
    
    boundingBoxes = [cv2.boundingRect(c) for c in contours]
    
    # sorting on x-axis 
    sortedByX = zip(*sorted(zip(contours, boundingBoxes),
    key=lambda b:b[1][0], reverse=x_reverse))
    
    # sorting on y-axis 
    (contours, boundingBoxes) = zip(*sorted(zip(*sortedByX),
    key=lambda b:b[1][1], reverse=y_reverse))

    # return the list of sorted contours and bounding boxes
    return (contours, boundingBoxes)

def get_rows(centers, row_amt, row_h):
    # Ann Zen:
    # https://stackoverflow.com/questions/66946804/python-sorting-items-from-top-left-to-bottom-right-with-opencv
    centers = np.array(centers)
    d = row_h / row_amt
    for i in range(row_amt):
        # We want to find all the points in the centers array that lies within the i row. 
        # The f = centers[:, 1] - d * i returns the centers array with each y coordinate 
        # subtracted by the distance between the top of the image and the top of the i row. 
        # So basically its like the i row got shifted upwards until it touched the top of the image.
        f = centers[:, 1] - d * i
        # With the shifted image, we can simple check if the points lie within the top of the image 
        # and the height of the row, hence the a = centers[(f < d) & (f > 0)].
        a = centers[(f < d) & (f > 0)]

        # The a.argsort(0) returns a's indices with its x coordinates and y coordinates sorted. 
        # Since we only want to sort the row by its x coordinates, we use the slice [:, 0], meaning all the rows at the 0 column. 
        # So a.argsort(0)[:, 0] is the array of indices, and yield a[a.argsort(0)[:, 0]] 
        # yields the rows sorted by the 0 columns. 
        # I realized that a[a.argsort(0)[:, 0]] can actually be replaced with a[a[:, 0].argsort()]
        yield a[a.argsort(0)[:, 0]]

def label_contour(image, c, index, color=(0, 255, 0), thickness=1):
    # compute the center of the contour area and draw a circle
    # representing the center
    M = cv2.moments(c)

    # add 1e-5 to avoid division by zero
    cX = int(M["m10"] / M["m00"] + 1e-5)
    cY = int(M["m01"] / M["m00"] + 1e-5)

    # draw the contour and label number on the image
    cv2.drawContours(image, [c], -1, color, thickness)
    cv2.putText(image, "#{}".format(index + 1), (cX - 20, cY), cv2.FONT_HERSHEY_PLAIN,
                1, (255, 0, 0), 1)

    # return the image with the contour number drawn on it
    return image

def label_contours(image, contours, color=(0, 255, 0), thickness=1):

    for (index, c) in enumerate(contours):
        label_contour(image, c, index, color, thickness)

    return image

def label_contour_cells(image, cells, color=(0, 255, 0), thickness=1):

    for (index, c) in enumerate(cells):
        c = c["cnt"]
        label_contour(image, c, index, color, thickness)

    return image

def label_centroid(image, x, y, index):

    # text green
    textColor=(255, 0, 0)

    # points blue
    pointColor=(0, 255, 0)

    # draw the centroid and label number on the image
    cv2.putText(image, "#{}".format(index + 1), (x, y), cv2.FONT_HERSHEY_PLAIN, 
                1, textColor, 1)
    cv2.circle(image, (x, y), 3, pointColor, -1)

    return image

def label_centroid_cells(image, cells):

    for (index, c) in enumerate(cells):
        pt = c["pos"]
        x = int(pt[0])
        y = int(pt[1])
        label_centroid(image, x, y, index)

    return image

def label_bounding_rect(image, rect, index):

    # text green
    textColor=(255, 0, 0)

    # rect color
    rectColor=(36, 255, 12)

    x,y,w,h = rect

    # draw the rectangle and label number on the image
    cv2.rectangle(image, (x, y), (x + w, y + h), rectColor, 1)

    cv2.putText(image, "#{}".format(index + 1), (x, y + 15), cv2.FONT_HERSHEY_PLAIN, 
                1, textColor, 1)

    return image

def label_bounding_cells(image, cells):

    for (index, c) in enumerate(cells):
        rect = c["rect"]
        label_bounding_rect(image, rect, index)

    return image


def find_cells(img, drawImage, Debug = False):
    """
    Find the cells of a grid
    """
    img_area = img.shape[0] * img.shape[1]

    (cnts, _) = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) # originally cv2.RETR_LIST

    # take the largest 200
    # cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:200]

    if (Debug):
        show_wait_destroy('all contours', cv2.drawContours(drawImage.copy(), cnts, -1, (0, 255, 0), 1))

    # Array containing the cropped cell image and its position in the grid
    cells = []
    for c in cnts:
        # Approximate the contour in order to determine whether the contour is a quadrilateral
        peri = cv2.arcLength(c, True)
        epsilon = 0.02 * peri # originally 0.017
        approx = cv2.approxPolyDP(c, epsilon, True)
        area = cv2.contourArea(approx)
        
        # https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html
        # 7.a. Straight Bounding Rectangle
        bounding_rect = cv2.boundingRect(approx) # bounding = (x,y,w,h)

        # 7.b. Rotated Rectangle
        rect = cv2.minAreaRect(approx)
        box = cv2.boxPoints(rect)
        box = np.int0(box) # convert into integer values = 4 x X and Y coordinates

        # We are looking for a contour of a specific area in relation to the grid size
        # and that is roughly quadrilateral
        # We filter for areas that are too small or too large in relation to the whole image
        percentage = (area * 100) / img_area
        if percentage > 0.01 and percentage < 2 and len(approx) == 4:
            # mask everything black except the contour (square) we are processing
            mask = np.zeros_like(img)
            cv2.drawContours(mask, [c], -1, 255, -1)
            # show_wait_destroy("mask", mask)

            # extract the coordinates for the white contour (square) we are processing
            (y, x) = np.where(mask == 255)
            (top_y, top_x) = (np.min(y), np.min(x))
            (bottom_y, bottom_x) = (np.max(y), np.max(x))
            cell = image[top_y : bottom_y + 1, top_x : bottom_x + 1]

            cell = cell.copy()
            # we crop the cell into its own 28 by 28 pixel image
            # cell = cv2.resize(cell, (28, 28))

            # We also find the centroid of the cell in relation
            # to the grid
            M = cv2.moments(c)

            # add 1e-5 to avoid division by zero
            cX = int(M["m10"] / M["m00"] + 1e-5)
            cY = int(M["m01"] / M["m00"] + 1e-5)

            cells.append(({"img": cell, "pos": (cX, cY), "cnt": c, "rect": bounding_rect, "box": box}))

            # show_wait_destroy("cell", cell)

    if (Debug):
        # select the cnt column and draw the selected contours
        cnts = [o["cnt"] for o in cells]
        show_wait_destroy('selected contours', cv2.drawContours(drawImage.copy(), cnts, -1, (0, 255, 0), 1))

        # select the cnt column and draw the rotated contours
        boxes = [o["box"] for o in cells]
        show_wait_destroy('rotated contours', cv2.drawContours(drawImage.copy(), boxes, -1, (0, 0, 255), 1))

        # label the selected contours
        labeled_cnts = label_contour_cells(drawImage.copy(), cells)
        show_wait_destroy('labeled contours', labeled_cnts)

        # label the centroids
        labeled_centroids = label_centroid_cells(drawImage.copy(), cells)
        show_wait_destroy("labeled centroids", labeled_centroids)

        # label the bounding rectangles
        labeled_bounding = label_bounding_cells(drawImage.copy(), cells)
        show_wait_destroy("labeled bounding", labeled_bounding)

    return cells

def get_grid(cells, drawImage, Debug = False):
    # https://stackoverflow.com/questions/38654302/how-can-i-sort-contours-from-left-to-right-and-top-to-bottom
    # # https://gist.githubusercontent.com/qgolsteyn/7da376ced650a2894c2432b131485f5d/raw/5a7b2e0150dfce942cc3cd1e28c3e2c8c0783936/main.py
    """
    Given a list of cells and they position, return a 2D array representing
    a grid, where each element of this 2D array contains of the value of the grid
    at that position.
    """
    grid = []

    rects = [o["rect"] for o in cells]
    r = np.array(rects) # have to convert to array to use np.max
    # max_height = np.max(r[::, 3]) # np.max(r[::, 3]) # index 3 is height => x, y, w, h
    max_height = 20
    debug('max_height', max_height)

    rects_unsorted = [o["pos"] for o in cells]
    rects_unsorted_array = np.array(rects_unsorted, dtype = np.float32)
    writeArrayToDisk(rects_unsorted_array, 'temp/rects_unsorted.txt')

    # Sort the contours by y-value
    # by_y = sorted(cells, key=lambda cell: cell["rect"][1])  # sort by index 1 which is y values => x, y, w, h
    by_y = sorted(cells, key=lambda cell: cell["pos"][1])  # sort by index 1 which is y values => x, y, w, h
    # writeListToDisk(by_y, 'temp/by_y.json')

    rects_sorted_by_y = [o["pos"] for o in by_y]
    rects_sorted_by_y_array = np.array(rects_sorted_by_y, dtype = np.float32)
    writeArrayToDisk(rects_sorted_by_y_array, 'temp/rects_sorted_by_y.txt')

    # if we are using the rects list  
    # sorted_rect_by_y = image.copy()
    # for (index, r) in enumerate(by_y):
    #     label_bounding_rect(sorted_rect_by_y, r, index)
    # show_wait_destroy('sorted by y', sorted_rect_by_y)

    # if we are using the full cells list  
    # label the bounding rectangles
    sorted_rect_by_y = label_bounding_cells(image.copy(), by_y)
    show_wait_destroy("sorted by y", sorted_rect_by_y)

    # line_y = by_y[0]["rect"][1]         # first y - used to be by_y[0][1] 
    line_y = by_y[0]["pos"][1]
    line = 1
    by_line = []

    # Assign a line number to each contour
    for c in by_y:
        # x, y, w, h = c["rect"]          # used to read the coordinates directly from by_y -> for x, y, w, h in by_y:
        x, y = c["pos"]
        w = h = max_height
        print('processing line number: ' + str(line) + ', line_y: ' + str(line_y) + ', x: ' + str(x) + ', y:' + str(y) + ', w:' + str(w) + ', h:' + str(h))
        if y > line_y + max_height:
            print('found new line since y ' + str(y) + ' > line_y ' + str(line_y) + ' + max_height ' + str(max_height))
            line_y = y
            line += 1
            print('new values. line: ' + str(line) + ', line_y: ' + str(line_y))
            
        by_line.append((line, x, y, w, h))

    # debug by_line array
    by_line_array = np.array(by_line, dtype = np.float32)
    writeArrayToDisk(by_line_array, 'temp/by_line.txt')

    # This will now sort automatically by line then by x
    rects_sorted = [(x, y, w, h) for (line, x, y, w, h) in sorted(by_line)]

    rects_sorted_array = np.array(rects_sorted, dtype = np.float32)
    writeArrayToDisk(rects_sorted_array, 'temp/rects_sorted.txt')

    sorted_rect = image.copy()
    for (index, r) in enumerate(rects_sorted):
        label_bounding_rect(sorted_rect, r, index)

    show_wait_destroy('sorted rect', sorted_rect)

    if (Debug):

        # select the position column
        centroids = [o["pos"] for o in cells]
        # for c in centroids:
        #     print('centroid %s' % str(c))

        sortedImage=drawImage.copy()
        h, w, c = sortedImage.shape
        count = 0
        for row in get_rows(centroids, 25, h):
            cv2.polylines(sortedImage, [row], False, (255, 0, 255), 1)
            for x, y in row:
                count += 1
                cv2.circle(sortedImage, (x, y), 5, (0, 0, 255), -1)  
                cv2.putText(sortedImage, str(count), (x - 10, y + 5), 1, cv2.FONT_HERSHEY_PLAIN, (0, 255, 255), 1)

        show_wait_destroy("sorted", sortedImage)

    return grid

image = cv2.imread('crossword3.png')

show_wait_destroy("raw image", image)

# Transform source image to gray if it is not already
if len(image.shape) != 2:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
else:
    gray = image

show_wait_destroy("gray image", gray)

grayOriginal = gray.copy();

# using a big blocksize seem to work well (blocksize = 51, c = 11)
thresh = cv2.adaptiveThreshold( 
    gray,
    maxValue=255.0,
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    thresholdType=cv2.THRESH_BINARY_INV,
    blockSize=11,
    C=2
)

# Show binary image
show_wait_destroy("thresh", thresh)

removeNoise(thresh, 400)
show_wait_destroy("thresh2", thresh)

# Fix horizontal and vertical lines (thickening)
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,3))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, vertical_kernel, iterations=1)

horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, horizontal_kernel, iterations=1)

show_wait_destroy("thresh3", thresh)

cells = find_cells(thresh, image, False)
# for cell in cells:
#     print('pre %s' % str(cell["pos"]))

# sort_rect_cells(cells)

# sort contours
# select the cnt column
# cnts = [o["cnt"] for o in cells]
# sorted_cntr, _ = sort_contours(cnts, x_axis_sort='LEFT_TO_RIGHT', y_axis_sort='TOP_TO_BOTTOM')
# labeled_sorted = label_contours(image.copy(), sorted_cntr)
# show_wait_destroy('sorted contours', labeled_sorted)

# sort cells by centroids
# sorted_cells = sorted(cells , key=lambda cell: [cell["pos"][1], cell["pos"][0]])
# for c in cells:
#     print('post %s' % str(c["pos"]))

# label the sorted cells
# labeled = label_centroid_cells(image.copy(), sorted_cells)
# show_wait_destroy("centroids sorted", labeled)

# cells = [o["img"] for o in cells]
# for index, c in enumerate(cells):
#     show_wait_destroy("cell", c)    

grid = get_grid(cells, image, True)


max_height: 20
processing line number: 1, line_y: 71, x: 556, y:71, w:20, h:20
processing line number: 1, line_y: 71, x: 664, y:73, w:20, h:20
processing line number: 1, line_y: 71, x: 629, y:74, w:20, h:20
processing line number: 1, line_y: 71, x: 520, y:81, w:20, h:20
processing line number: 1, line_y: 71, x: 483, y:85, w:20, h:20
processing line number: 1, line_y: 71, x: 667, y:108, w:20, h:20
found new line since y 108 > line_y 71 + max_height 20
new values. line: 2, line_y: 108
processing line number: 2, line_y: 108, x: 633, y:109, w:20, h:20
processing line number: 2, line_y: 108, x: 597, y:111, w:20, h:20
processing line number: 2, line_y: 108, x: 524, y:115, w:20, h:20
processing line number: 2, line_y: 108, x: 486, y:119, w:20, h:20
processing line number: 2, line_y: 108, x: 702, y:144, w:20, h:20
found new line since y 144 > line_y 108 + max_height 20
new values. line: 3, line_y: 144
processing line number: 3, line_y: 144, x: 637, y:145, w:20, h:20
processing line number: 3, 